In [1]:
!pip install websocket-client

     |████████████████████████████████| 61kB 3.0MB/s 


In [2]:
# from https://stackoverflow.com/questions/49648391/how-to-install-ta-lib-in-google-colab

url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 5.1MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp37-cp37m-linux_x86_64.whl size=1437996 sha256=f3d9501ecf3f09da528ffeff8bf74a5c2a352e62584f1e4308d274db602daaaf
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [3]:
!pip install python-binance

     |████████████████████████████████| 51kB 2.8MB/s 
     |████████████████████████████████| 286kB 6.9MB/s 
     |████████████████████████████████| 3.2MB 16.5MB/s 
     |████████████████████████████████| 3.1MB 47.3MB/s 
     |████████████████████████████████| 184kB 48.2MB/s 
     |████████████████████████████████| 501kB 52.3MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 256kB 44.4MB/s 


In [4]:
import websocket, json, pprint, talib, numpy

In [5]:
from binance.client import Client
from binance.enums import *

In [6]:
SOCKET = "wss://stream.binance.com:9443/ws/ethusdt@kline_1m"

In [7]:
def on_open(ws):
    print('opened connection')

In [8]:
def on_close(ws):
    print('closed connection')

In [13]:
def on_message(ws, message):
    print('received message')
    print(message)
    json_message = json.loads(message)
    pprint.pprint(json_message)

In [ ]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

In [11]:
RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05

closes = []
in_position = False

def on_message(ws, message):
    global closes, in_position
    
    #print('received message')
    json_message = json.loads(message)
    #pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        pprint.pprint(json_message)
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

In [12]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

opened connection
closed connection


False

In [ ]:
RSI_PERIOD = 14
RSI_OVERBOUGHT = 70
RSI_OVERSOLD = 30
TRADE_SYMBOL = 'ETHUSD'
TRADE_QUANTITY = 0.05

closes = []
in_position = False

def on_message(ws, message):
    global closes, in_position
    
    print('received message')
    json_message = json.loads(message)
    pprint.pprint(json_message)

    candle = json_message['k']

    is_candle_closed = candle['x']
    close = candle['c']

    if is_candle_closed:
        pprint.pprint(json_message)
        print("candle closed at {}".format(close))
        closes.append(float(close))
        print("closes")
        print(closes)

        if len(closes) > RSI_PERIOD:
            np_closes = numpy.array(closes)
            rsi = talib.RSI(np_closes, RSI_PERIOD)
            print("all rsis calculated so far")
            print(rsi)
            last_rsi = rsi[-1]
            print("the current rsi is {}".format(last_rsi))

            if last_rsi > RSI_OVERBOUGHT:
                if in_position:
                    print("Overbought! Sell! Sell! Sell!")
                    # put binance sell logic here
                    order_succeeded = True
                    if order_succeeded:
                        in_position = False
                else:
                    print("It is overbought, but we don't own any. Nothing to do.")
            
            if last_rsi < RSI_OVERSOLD:
                if in_position:
                    print("It is oversold, but you already own it, nothing to do.")
                else:
                    print("Oversold! Buy! Buy! Buy!")
                    # put binance buy order logic here
                    if order_succeeded:
                        in_position = True

In [ ]:
ws = websocket.WebSocketApp(SOCKET, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()

Streaming output truncated to the last 5000 lines.
       'q': '381639.55789580',
       's': 'ETHUSDT',
       't': 1619673420000,
       'v': '141.38958000',
       'x': False},
 's': 'ETHUSDT'}
received message
{"e":"kline","E":1619673462835,"s":"ETHUSDT","k":{"t":1619673420000,"T":1619673479999,"s":"ETHUSDT","i":"1m","f":382154668,"L":382154939,"o":"2698.01000000","c":"2698.12000000","h":"2700.64000000","l":"2698.00000000","v":"157.89420000","n":272,"x":false,"q":"426178.67782640","V":"85.51058000","Q":"230787.60931820","B":"0"}}
{'E': 1619673462835,
 'e': 'kline',
 'k': {'B': '0',
       'L': 382154939,
       'Q': '230787.60931820',
       'T': 1619673479999,
       'V': '85.51058000',
       'c': '2698.12000000',
       'f': 382154668,
       'h': '2700.64000000',
       'i': '1m',
       'l': '2698.00000000',
       'n': 272,
       'o': '2698.01000000',
       'q': '426178.67782640',
       's': 'ETHUSDT',
       't': 1619673420000,
       'v': '157.89420000',
       'x': Fals